In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample


In [10]:

# Load and combine datasets
df1 = pd.read_csv('pdmm_optimization_results_2.csv')
df2 = pd.read_csv('pdmm_optimization_results_3.csv')
df = pd.concat([df1, df2], ignore_index=True)

# Drop irrelevant columns  and prepare data
df = df.drop(columns=['Final_Error'])  # Keep only features and target
X = df[['Timer','K_decision', 'Rejection_Threshold']].values
y = df['Converged'].astype(int).values  # Convert boolean to 0/1

# Handle class imbalance (upsample minority class)
df_majority = df[df['Converged'] == False]
df_minority = df[df['Converged'] == True]
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)
df_balanced = pd.concat([df_majority, df_minority_upsampled])

X = df_balanced[['Timer','K_decision', 'Rejection_Threshold']].values
y = df_balanced['Converged'].astype(int).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:

# Build neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])

# Train model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, batch_size=32, verbose=1)

# Evaluate
test_loss, test_acc, test_auc = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"\nTest Accuracy: {test_acc:.4f}, AUC: {test_auc:.4f}")


Epoch 1/100


c:\Users\za_re\anaconda3\envs\sklearn-env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6835 - auc_2: 0.7549 - loss: 0.6425 - val_accuracy: 0.8706 - val_auc_2: 0.8855 - val_loss: 0.5760
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8022 - auc_2: 0.8680 - loss: 0.5638 - val_accuracy: 0.8941 - val_auc_2: 0.9083 - val_loss: 0.4963
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8841 - auc_2: 0.9041 - loss: 0.4845 - val_accuracy: 0.8941 - val_auc_2: 0.9110 - val_loss: 0.4339
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8576 - auc_2: 0.8906 - loss: 0.4384 - val_accuracy: 0.8941 - val_auc_2: 0.9074 - val_loss: 0.3864
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8860 - auc_2: 0.9273 - loss: 0.3770 - val_accuracy: 0.8941 - val_auc_2: 0.9016 - val_loss: 0.3508
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8770 - auc_2: 0.9287 - loss: 0.3495 - val_accuracy: 0.8941 - val_auc_2: 0.8991 - val_loss: 0.3260
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━

In [14]:

# Predict convergence for new parameters
def predict_convergence(timer,k_decision, rejection_threshold):
    input_data = np.array([[timer,k_decision, rejection_threshold]])
    scaled_input = scaler.transform(input_data)
    prob = model.predict(scaled_input)[0][0]
    return prob >= 0.5  # Threshold at 0.5



In [13]:
# Generate a grid of parameters to explore
t_values = np.arange(1, 5) 
k_values = np.arange(1, 5)  # K_decision ranges from 1-4
thresholds = np.linspace(1, 15, 50)  # Thresholds from 1 to 15

converged_params = []
for t in t_values:
    for k in k_values:
        for th in thresholds:
            if predict_convergence(t, k, th):
                converged_params.append((t, k, th))
                
# Convert to a NumPy array, find unique rows
unique_params = np.unique(converged_params, axis=0)

# Print them if you want
print("Suggested parameters for convergence:")
print(unique_params)

# Save to CSV
df_params = pd.DataFrame(unique_params, columns=['Timer', 'K_decision', 'Threshold'])
df_params.to_csv('converged_params.csv', index=False)

print("Saved converged_params.csv with columns [Timer, K_decision, Threshold].")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━